In [1]:
import pandas as pd
import numpy as np

In [63]:
df

,viewers,streamer,counts
0,00000kim,Kamet0,16
1,0000_89,AntoineDaniel,17
2,0000kaizer0000,Kamet0,38
3,0000kaizer0000,Kamet0,16
4,0000kaizer0000,Ponce,13
...,...,...,...
2117835,zzzzzzzz74,ZeratoR,11
2117836,zzzzzzzz74,ZeratoR,45
2117837,zzzzzzzz74,otplol_,27
2117838,zzzzzzzz74,samueletienne,19


In [59]:
grouped = df.groupby('viewers')['streamer'].nunique()

# filter out the viewers that have only one unique streamer
filtered = grouped[grouped > 1]

# print the resulting dataframe
print(filtered)

viewers
0000kaizer0000           4
000adzijl                2
000good                  3
000hiro                  3
000inaso                 2
                        ..
zzzz802                  3
zzzzing                  2
zzzzzaaaaazzzzzaaaaa     3
zzzzzza                  2
zzzzzzzz74              10
Name: streamer, Length: 352907, dtype: int64


In [60]:
filtered_df = df[df['viewers'].isin(filtered.index)]

In [62]:
filtered_df

,viewers,streamer,counts
2,0000kaizer0000,Kamet0,38
3,0000kaizer0000,Kamet0,16
4,0000kaizer0000,Ponce,13
5,0000kaizer0000,Sardoche,16
6,0000kaizer0000,otplol_,31
...,...,...,...
2117834,zzzzzzzz74,ZeratoR,19
2117835,zzzzzzzz74,ZeratoR,11
2117836,zzzzzzzz74,ZeratoR,45
2117837,zzzzzzzz74,otplol_,27


In [55]:
# remove the users if they only watch one streamer


,viewers,streamer,counts
2,0000kaizer0000,Kamet0,38
3,0000kaizer0000,Kamet0,16
7,0001t,Domingo,15
8,0001t,Domingo,16
10,000adzijl,aminematue,12
...,...,...,...
2117829,zzzzzzzz74,Kamet0,22
2117830,zzzzzzzz74,Kamet0,11
2117834,zzzzzzzz74,ZeratoR,19
2117835,zzzzzzzz74,ZeratoR,11


In [2]:
def similarity(user1, user2):
    # calculate the similarity between two users
    # return the similarity
    return np.dot(user1, user2) / (np.linalg.norm(user1) * np.linalg.norm(user2))

In [3]:
def recommandation(matrix, user):
    # calculate the recommandation for a user
    return np.dot(matrix, user) / np.linalg.norm(user)

In [4]:
df = pd.read_csv('graph/final_fr.csv')

In [5]:
# create a pivot table for streamers and addition the counts
df_pivot_streamer = df.pivot_table(index='viewers', columns='streamer', values='counts', fill_value=0, aggfunc='sum')
df_pivot_category = df.pivot_table(index='viewers', columns='category', values='counts', fill_value=0, aggfunc='sum')

In [6]:
df_pivot_streamer.fillna(0, inplace=True)
df_pivot_category.fillna(0, inplace=True)

In [7]:
matrix_streamer = df_pivot_streamer.to_numpy()
matrix_category = df_pivot_category.to_numpy()

In [36]:
user = "yayamontagne"

In [9]:
# show the user streamers
user_streamers = df_pivot_streamer.loc[user]
user_streamers = user_streamers[user_streamers > 0]
print(user_streamers)

streamer
1PVCS             11
Chap_GG           14
Domingo           14
Gotaga            17
Kamet0            23
Nisqyy            16
RocketBaguette    13
Sardoche          15
otplol_           89
Name: yayalfc, dtype: int64


In [10]:
# show the user categories
user_categories = df_pivot_category.loc[user]
user_categories = user_categories[user_categories > 0]
print(user_categories)

category
Counter-Strike: Global Offensive     11
Just Chatting                        52
League of Legends                   119
Rocket League                        13
Special Events                       17
Name: yayalfc, dtype: int64


In [46]:
def recommand_category(user):
    # get the user index
    user_index = df_pivot_category.index.get_loc(user)
    # get the user's categories
    user_categories = df_pivot_category.loc[user]
    user_categories = user_categories[user_categories > 0]
    # get the user
    user = matrix_category[user_index]
    # calculate the similarity between the user and all the other users
    similarities = []
    for i in range(len(matrix_category)):
        similarities.append(similarity(user, matrix_category[i]))
    # get the most similar users
    most_similar_users = np.argsort(similarities)[::-1][:5000]
    # naive recommandation system that loops through the most similar users and add the categories to a list
    recommandation = {}
    for i in most_similar_users:
        # get the categories of the user
        categories = df_pivot_category.iloc[i]
        categories = categories[categories > 0]
        # add the categories to the recommandation list
        for category in categories.index:
            # add counts to the category key
            recommandation[category] = recommandation.get(category, 0) + categories[category]
    # filter out the categories that the user already follows
    for category in user_categories.index:
        if category in recommandation:
            del recommandation[category]
    # get the top 10 recommandation
    top_10_recommandation = sorted(recommandation.items(), key=lambda x: x[1], reverse=True)[:10]
    # print the top 10 recommandation
    for i, reco in enumerate(top_10_recommandation):
        print(f'#{i} - {reco[0]}')

In [47]:
recommand_category(user)

#0 - Pummel Party
#1 - Trackmania
#2 - Tabletop RPGs
#3 - Elden Ring
#4 - Trigon: Space Story
#5 - Rogue Legacy 2
#6 - Worms W.M.D
#7 - Talk Shows & Podcasts
#8 - Minecraft
#9 - Overwatch


In [44]:
def recommand_streamer(user):
    # get the user index
    user_index = df_pivot_streamer.index.get_loc(user)
    # retrieve the user's streamers
    user_streamers = df_pivot_streamer.iloc[user_index]
    user_streamers = user_streamers[user_streamers > 0]
    # get the user
    user = matrix_streamer[user_index]
    # calculate the similarity between the user and all the other users
    similarities = []
    for i in range(len(matrix_streamer)):
        similarities.append(similarity(user, matrix_streamer[i]))
    # get the most similar users
    most_similar_users = np.argsort(similarities)[::-1][:5000]
    # naive recommandation system that loops through the most similar users and add the streamers to a list
    recommandation = {}
    for i in most_similar_users:
        # get the streamers of the user
        streamers = df_pivot_streamer.iloc[i]
        streamers = streamers[streamers > 0]
        # add the streamers to the recommandation list
        for streamer in streamers.index:
            # add counts to the streamer key
            recommandation[streamer] = recommandation.get(streamer, 0) + streamers[streamer]
    # filter out the streamers that the user already follows
    for streamer in user_streamers.index:
        if streamer in recommandation:
            del recommandation[streamer]
    # get the top 10 recommandation
    top_10_recommandation = sorted(recommandation.items(), key=lambda x: x[1], reverse=True)[:10]
    # print the top 10 recommandation
    for i, reco in enumerate(top_10_recommandation):
        print(f'#{i} - {reco[0]}')

In [45]:
recommand_streamer(user)

#0 - ZeratoR
#1 - Ponce
#2 - Mynthos
#3 - Joueur_du_Grenier
#4 - Etoiles
#5 - JeanMassiet
#6 - AlphaCast
#7 - Clemovitch
#8 - Kamet0
#9 - samueletienne
